In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv('/kaggle/input/forexsoftware-historical-forex-data/Historical Forex Data/EURUSD_M1.csv')

In [13]:
df.head()

,Time,Open,High,Low,Close,Volume
0,30/05/2024 9:16,1.08171,1.08181,1.08167,1.08180,135
1,30/05/2024 9:17,1.08181,1.08181,1.08166,1.08166,27
2,30/05/2024 9:18,1.08167,1.08172,1.08167,1.08171,57
3,30/05/2024 9:19,1.08169,1.08169,1.08159,1.08161,114
4,30/05/2024 9:20,1.08162,1.08163,1.08152,1.08158,109


In [14]:
df.describe()

,Open,High,Low,Close,Volume
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.088571,1.088621,1.088520,1.088571,146.344730
std,0.013323,0.013326,0.013321,0.013323,159.493171
min,1.066710,1.066820,1.066600,1.066700,1.000000
25%,1.078940,1.078980,1.078890,1.078940,46.000000
50%,1.086820,1.086870,1.086760,1.086820,100.000000
75%,1.095200,1.095270,1.095110,1.095200,190.000000
max,1.120070,1.120150,1.120020,1.120080,2174.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    100000 non-null  object 
 1   Open    100000 non-null  float64
 2   High    100000 non-null  float64
 3   Low     100000 non-null  float64
 4   Close   100000 non-null  float64
 5   Volume  100000 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB


In [16]:
df.isna().sum()

Time      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [17]:
df.isnull().sum()

Time      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

## Visualization

In [ ]:
plt.figure(figsize=(12,8))

plt.plot(df['Time'], df['Open'], label='Open', marker='o')
plt.plot(df['Time'], df['High'], label='High', marker='o')
plt.plot(df['Time'], df['Low'], label='Low', marker='o')
plt.plot(df['Time'], df['Close'], label='Close', marker='o')

plt.title('Time Series of Open, High, Low, Close Prices')
plt.xlabel('Time')
plt.ylabel('Prices')
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Candlestick Chart')
sns.barplot(x=df['Time'], y=df['Open'], data=df, label='Open',color='lightgreen')


**Creating more features**

In [18]:
# df['Next_Close'] = df['Close'].shift(-1)

TypeError: 'DataFrame' object is not callable